In [6]:
from math import factorial, gcd,sqrt

'''
http://robin.pollak.io/wizard_factoring.pdf
'''

pq1 = (4715832397,5476546811)
n1 = pq1[0]*pq1[1]

pq2 = (47217592089965824021278357521901144689506741271419, 19244123869039008990587632107835659152065053045809)
n2  = pq2[0] * pq2[1] 

pq3 = (9748732579,6892276889)
n3 = pq3[0] * pq3[1] 

pq1 = (471841,270689)
n1 = pq1[0]*pq1[1]
n4 = 1035896947 *3500688283
n = 11321*22943
#n = 94343*23143
#n = 24430436836352483797063096106414774550335403158860524990373856944254297586018983298705492233654830523199692736514219239319146510300625339876324647894404893431924815784960576423322270642064447439532880084271906506181596791330255472317736274838975886982878636272375371266076163692482358755032762905350584467818322896256697824686362875462689754439034305568485764372282041358336076968265549938271297558180529807346415680592340188076635410423523198461463168140415659131710799290510682225326203262832075315314659506065014839527357257670962979561415126272979988642044180466409976304796211964778255235614585976098809592350027




#n =n4
a = 2
i = 2
d = 1

In [105]:
for a in range(0,n):
    i = 2
    d = 1
    # Pollard is fast for values of p which has small factors
    # So I decided to limit the search till 10
    while d<=1 and i<7:
        f= (a**factorial(i)-1)%n
        d = gcd(f,n)
        #print((a**factorial(i)-1)%n)
        i+=1
    if d > 1 and d != n:
        p = d
        q = int(n/d)
        print(f"p: {p} and q: {q} || a : {a} ")
        break

KeyboardInterrupt: 

In [10]:
from itertools import count, compress, takewhile, cycle
from math import gcd, fsum, log
try:
    from gmpy2 import mpz
except ImportError:
    mpz = int

def prime_sieve(start=1, end=float('inf')): # postponed sieve, by Will Ness
    for c in (2,3,5,7):                     # original code David Eppstein,
        if start <= c <= end:               # start and end added by Oscar Smith
            yield c
        elif end < c:
            return
    sieve = {}                              # Alex Martelli, ActiveState Recipe 2002
    ps = prime_sieve()                      # a separate base Primes Supply:
    p = next(ps) and next(ps)               # (3) a Prime to add to dict
    q = p*p                                 # (9) its sQuare
    for c in count(9,2):                    # the Candidate
        if c in sieve:                      # c’s a multiple of some base prime
            s = sieve.pop(c)                # i.e. a composite ; or
        elif c < q:
            if start <= c <= end:
                yield c                     # a prime
                continue
            elif end < c:
                return
        else:   # (c==q):            # or the next base prime’s square:
            s=count(q+2*p,2*p)       #    (9+6, by 6 : 15,21,27,33,...)
            p=next(ps)               #    (5)
            q=p*p                    #    (25)
        for m in s:                  # the next multiple
            if m not in sieve:       # no duplicates
                break
        sieve[m] = s                 # original test entry: ideone.com/WFv4f

def mod_mersenne(n, prime, mersenne):
    """ Calculates n % 2^prime-1 where mersenne_prime=2**prime-1 """
    while n > mersenne:
        n = (n & mersenne) + (n >> prime)
    return n if n != mersenne else 0

def pow_mod_mersenne(base, exp, prime, mersenne):
    """ Calculates base^exp % 2^prime-1 where mersenne_prime=2**prime-1 """
    number = 1
    while exp:
        if exp & 1:
            number = mod_mersenne(number * base, prime, mersenne)
        exp >>= 1
        base = mod_mersenne(base * base, prime, mersenne)
    return number

def p_minus1(prime, mersenne, B1, B2):
    """ Does 2**prime-1 have a factor 2*k*prime+1?
        such that the largest prime factor of k is less than limit"""
    log_B1 = log(B1)
    M = mpz(1)
    for p in prime_sieve(end=B1):
        M *= p**int(log_B1/log(p))
    M = pow_mod_mersenne(3, 2*M*prime, prime, mersenne)
    g = gcd(mersenne, M-1)
    if 1 < g < mersenne:
        return True
    if g == mersenne:
        return False
    return p_minus1_stage_2(prime, mersenne, M, p, B2)


def p_minus1_stage_2(prime, mersenne, M, B1, B2):
    delta_cache = {0:M}
    delta_max = 0
    S = mod_mersenne(M*M, prime, mersenne)
    p_old = B1
    HQ = M
    for k, p in enumerate(prime_sieve(start=B1, end=B2)):
        delta = p - p_old
        if delta not in delta_cache:
            for d in range(delta_max, delta, 2):
                delta_cache[d+2] = mod_mersenne(delta_cache[d] * S, prime, mersenne)
                delta_max = delta
        HQ = mod_mersenne(HQ*delta_cache[delta], prime, mersenne)
        M = mod_mersenne(M*(HQ-1), prime, mersenne)
        p_old = p
        if k % 100 == 0:
            g = gcd(M, mersenne)
            if 1 < g < mersenne:
                return True
            if g == mersenne:
                return False
    return 1 < gcd(M, mersenne) < mersenne

if __name__ == '__main__':
    primes = prime_sieve()
    next(primes)
    for prime in primes:
        B1 = int(10*prime.bit_length())
        p_minus1(prime, 2**mpz(prime)-1, B1, 10*B1)
        if prime>2000:
             break

In [15]:
def sieve_primes(num):
    l = list(range(2,num+1))
    indx = 0
    while l[indx]<=int(num**0.5):       # untill the square of the number +1
        for n  in l[indx+1:]:
            if n%l[indx] == 0:
                l.remove(n)
        indx+=1
    return l
def primes(n):
    ps = []
    sieve = [True] * n
    for p in range(2,n):
        if sieve[p]:
            ps.append(p)
            for i in range(p*p,n,p):
                sieve[i] = False
    return ps
def primes(n):
    ps = []
    sieve = [True] * n
    for p in range(2,n):
        if sieve[p]:
            ps.append(p)
            for i in range(p*p,n,p):
                sieve[i] = False
    return ps

In [16]:
def pollardp_1(n,b):
    c = 2
    for p in primes(b):
        pp = p
        while pp<b:
            c = pow(c,p,n)
            pp *= p
    p = gcd(c-1,n)
    if 1<p<n:
        return p
    else:
        print("Fail")

In [21]:
pollardp_1(n4,math.sq)

MemoryError: 